### Limpieza y análisis dataset 3

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df3 = pd.read_csv('../data/Raw/Study_summary_human.csv')
df3.shape

(5968, 67)

In [3]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5968 entries, 0 to 5967
Data columns (total 67 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   source                                5968 non-null   object 
 1   DOI                                   5968 non-null   object 
 2   Authors                               5968 non-null   object 
 3   Year                                  5968 non-null   int64  
 4   Species                               5968 non-null   object 
 5   Life Stage                            5968 non-null   object 
 6   In vitro/in vivo                      5968 non-null   object 
 7   Sex                                   4383 non-null   object 
 8   Experiment Type                       5968 non-null   object 
 9   Exposure Route                        5968 non-null   object 
 10  Particle Mix?                         5968 non-null   object 
 11  Negative Control 

In [4]:
print("LISTA COMPLETA DE VARIABLES")
for i, columna in enumerate(df3.columns):
    print(f"{i+1:2d}. {columna}")

LISTA COMPLETA DE VARIABLES
 1. source
 2. DOI
 3. Authors
 4. Year
 5. Species
 6. Life Stage
 7. In vitro/in vivo
 8. Sex
 9. Experiment Type
10. Exposure Route
11. Particle Mix?
12. Negative Control
13. Reference Particle
14. Exposure Media
15. Solvent
16. Detergent
17. pH
18. Salinity (ppt)
19. Temperature (Avg)
20. Temperature (Min)
21. Temperature (Max)
22. Exposure Duration (days)
23. Number of Doses
24. Replicates
25. Sample Size
26. Dosing Frequency
27. Nominal Chemical Added
28. Nominal Chemical Added Dose
29. Nominal Chemical Added Dose Units
30. Measured Chemical Added
31. Measured Chemical Added Dose
32. Measured Chemical Added Dose Units
33. Selected Dose
34. Effect
35. Direction
36. Broad Endpoint Category
37. Specific Endpoint Category
38. Endpoint
39. Level of Biological Organization
40. Target Organelle, Cell, or Tissue
41. Polymer
42. Shape
43. Density (g/cm^3)
44. Density, reported or estimated
45. Charge
46. Zeta Potential (mV)
47. Zeta Potential Media
48. Function

In [5]:
# Primera limpieza de columnas que no aportan al modelo
df3 = df3.drop(['source',
                'Authors',
                'Temperature (Min)', 
                'Temperature (Max)', 
                'Nominal Chemical Added', 
                'Nominal Chemical Added Dose',
                'Nominal Chemical Added Dose Units', 
                'Measured Chemical Added',   
                'Measured Chemical Added Dose',   
                'Measured Chemical Added Dose Units', 
                'Direction'], axis = 1)

In [6]:
# Columnas duplicadas
duplicadas = df3.columns[df3.T.duplicated()]
duplicadas

Index([], dtype='object')

In [7]:
df3.duplicated().sum()

np.int64(1386)

In [8]:
df3 = df3.drop_duplicates().reset_index(drop=True)
df3.shape

(4582, 56)

#### Tratamiento de nulos

In [9]:
def inspect_unique_values(df, max_unique=30):

    print("\n============================")
    print(" INSPECCIÓN DE COLUMNAS POR TIPO ")
    print("============================\n")

    for col in df.columns:
        uniques = df[col].dropna().unique()
        num_unique = len(uniques)

        # ¿Es object?
        if df[col].dtype == "object":
            print(f"\n🔵 OBJECT — {col} ({num_unique} categorías)")
            print(sorted(uniques[:max_unique]))
        
        # ¿Es boolean?
        elif df[col].dtype == "bool":
            print(f"\n🟢 BOOLEAN — {col} ({num_unique} valores)")
            print(sorted(uniques))

        # ¿Es numérica con pocas categorías?
        elif pd.api.types.is_numeric_dtype(df[col]) and num_unique < 20:
            print(f"\n🟣 NUMÉRICA-CATEGÓRICA — {col} ({num_unique} valores)")
            print(sorted(uniques))

        # Si no es categórica, no mostramos nada
        else:
            continue

        print("----------------------------------------")


inspect_unique_values(df3)



 INSPECCIÓN DE COLUMNAS POR TIPO 


🔵 OBJECT — DOI (65 categorías)
['10.1002/jbm.a.30354', '10.1002/jbmr.5650080907', '10.1002/pat.3749', '10.1007/s00204-019-02478-7', '10.1007/s00204-020-02750-1', '10.1007/s10904-013-9856-3', '10.1007/s12013-015-0705-6', '10.1016/0142-9612(96)83279-0', '10.1016/S0142-9612(01)00290-3', '10.1016/j.biomaterials.2011.07.037', '10.1016/j.biomaterials.2016.01.064', '10.1016/j.chemosphere.2017.11.076', '10.1016/j.envres.2017.08.043', '10.1016/j.jhazmat.2018.06.017', '10.1016/j.jhazmat.2019.121575', '10.1016/j.scitotenv.2018.08.353', '10.1016/j.scitotenv.2019.05.071', '10.1016/j.scitotenv.2020.138183', '10.1016/j.tiv.2019.104610', '10.1038/srep46687', '10.1039/c7an00461c', '10.1039/c9en00523d', '10.1039/c9tx00147f', '10.1046/j.1525-1594.2003.07107.x', '10.1067/mlc.2001.114677', '10.1080/17435390.2016.1218080', '10.1166/jbn.2006.004', '10.1166/jnn.2019.16347', '10.2147/IJN.S161369', '10.7546/CRABS.2018.10.07']
----------------------------------------

🟣 NUMÉR

In [10]:
# Columnas binarias detectadas que requieren normalización
binary_cols = [
    'Size Validated?',
    'Polymer Validated?',
    'Shape Validated',
    'Sodium Azide Present?',
    'Screened for Chemical Contamination?',
    'Particle Cleaning?',
    'Solvent Rinse',
    'Background Contamination Monitored?',
    'Concentration Validated?',
    'Uptake Validated?',
    'Organisms Fed?',
]

# Diccionario de reemplazo universal
binary_map = {
    'y': 'yes', 'yes': 'yes', 'true': 'yes', '1': 'yes',
    'n': 'no', 'no': 'no', 'false': 'no', '0': 'no',
    'unknown': 'unknown', 'not_reported': 'unknown', 'none': 'unknown',
}

# Limpieza para cada columna
for col in binary_cols:
    if col in df3.columns:

        # Pasar todo a string
        df3[col] = df3[col].astype(str).str.strip().str.lower()

        # Reemplazar valores usando binary_map
        df3[col] = df3[col].replace(binary_map)

        # Los valores que no encajen los convertimos a 'unknown'
        df3[col] = df3[col].where(
            df3[col].isin(['yes', 'no', 'unknown']),
            'unknown'
        )

        # Convertir 'nan' textual a NA real
        df3[col] = df3[col].replace('nan', np.nan)

        print(f"✔ Normalizada: {col}")

print("\nNormalización completada.")

✔ Normalizada: Size Validated?
✔ Normalizada: Polymer Validated?
✔ Normalizada: Shape Validated
✔ Normalizada: Sodium Azide Present?
✔ Normalizada: Screened for Chemical Contamination?
✔ Normalizada: Particle Cleaning?
✔ Normalizada: Solvent Rinse
✔ Normalizada: Background Contamination Monitored?
✔ Normalizada: Concentration Validated?
✔ Normalizada: Uptake Validated?
✔ Normalizada: Organisms Fed?

Normalización completada.


In [11]:
sensitive_cols = [
    "Functional Group",
    "Target Organelle, Cell, or Tissue",
    "Zeta Potential Media"
]

na_tokens = ["not_reported", "nr", "nan", ".", "-", ""]

for col in sensitive_cols:
    if col in df3.columns:
        df3[col] = df3[col].replace(na_tokens, np.nan)
        print(f"✔ Limpieza NA aplicada (columna sensible): {col}")

print("\nLimpieza de columnas sensibles completada.\n")

✔ Limpieza NA aplicada (columna sensible): Functional Group
✔ Limpieza NA aplicada (columna sensible): Target Organelle, Cell, or Tissue
✔ Limpieza NA aplicada (columna sensible): Zeta Potential Media

Limpieza de columnas sensibles completada.



In [12]:
from IPython.display import display
#nulos
with pd.option_context('display.max_rows', None):
    display(df3.isna().mean().sort_values(ascending=False) * 100)

Salinity (ppt)                          100.000000
pH                                       99.563509
Functional Group                         78.830205
Zeta Potential Media                     72.304670
Tissue Distribution                      71.780882
Zeta Potential (mV)                      71.388040
Charge                                   69.161938
Temperature (Avg)                        64.709734
Selected Dose                            34.242689
Sex                                      28.415539
Particle Mass (mg)                        8.402444
Particle Volume (μm^3)                    7.835007
Target Organelle, Cell, or Tissue         3.230031
Weathered or Biofouled?                   2.073330
Density, reported or estimated            1.374945
Particle Length (μm)                      0.414666
Exposure Media                            0.414666
Replicates                                0.283719
Density (g/cm^3)                          0.240070
Sample Size                    

In [13]:
df3 = df3.drop(['Salinity (ppt)', 'pH', 'Tissue Distribution'], axis = 1)

#### Variables a imputar los Nan

In [14]:
df3["Species"].unique()

array(['(Rat) Rattus norvegicus', '(Human) Homo sapiens',
       '(Mouse) Mus musculus', '(Sheep) Ovis aries',
       '(Pig) Sus domesticus'], dtype=object)

In [15]:
df3["Functional Group"] = df3["Functional Group"].fillna("not_reported")

print("✔ Functional Group: NA imputado como 'not_reported'")


✔ Functional Group: NA imputado como 'not_reported'


In [16]:
df3["Zeta Potential Media"] = df3["Zeta Potential Media"].fillna("not_reported")

print("✔ Zeta Potential Media: NA imputado como 'not_reported'")

✔ Zeta Potential Media: NA imputado como 'not_reported'


In [17]:
# Zeta Potential (mV) - imputación por polymer, fallback global

col1 = 'Zeta Potential (mV)'
group_col1 = 'Polymer'

# Forzar a numérico
df3[col1] = pd.to_numeric(df3[col1], errors='coerce')
df3[group_col1] = df3[group_col1].astype(str).str.strip()

# Flag missing original
df3['zeta_missing'] = df3[col1].isna().astype(int)

# Mediana por polymer
medians_by_poly = df3.groupby(group_col1)[col1].median()

# Mapeo auxiliar
df3['zeta_median_by_poly'] = df3[group_col1].map(medians_by_poly)

# Mediana global
global_median = df3[col1].median()

# Imputación final
df3[col1] = df3[col1].fillna(df3['zeta_median_by_poly'])
df3[col1] = df3[col1].fillna(global_median)

# Flags de trazabilidad
df3['zeta_imputed_by_group'] = ((df3['zeta_missing'] == 1) &
                               df3['zeta_median_by_poly'].notna()).astype(int)

df3['zeta_imputed_by_global'] = ((df3['zeta_missing'] == 1) &
                                df3['zeta_median_by_poly'].isna()).astype(int)

# Limpiar aux
df3 = df3.drop(columns=['zeta_median_by_poly'])

# Resumen
print("=== Imputación Zeta Potential (mV) ===")
print("Total filas:", len(df3))
print("Missing iniciales:", df3['zeta_missing'].sum())
print("Imputadas por mediana polymer:", df3['zeta_imputed_by_group'].sum())
print("Imputadas por mediana global:", df3['zeta_imputed_by_global'].sum())
print("Missing tras imputación:", df3[col1].isna().sum())


=== Imputación Zeta Potential (mV) ===
Total filas: 4582
Missing iniciales: 3271
Imputadas por mediana polymer: 2508
Imputadas por mediana global: 763
Missing tras imputación: 0


In [18]:
df3["charge_missing"] = df3["Charge"].isna().astype(int)
df3["Charge"] = df3["Charge"].fillna("unknown")

print("✔ Charge imputado con 'unknown'")

✔ Charge imputado con 'unknown'


In [19]:
col = "Temperature (Avg)"
mode_col = "In vitro/in vivo"   # nombre correcto en este dataset

# Normalizar In vitro/in vivo
df3["in_vitro_flag"] = (
    df3[mode_col]
    .astype(str)
    .str.strip()
    .str.lower()
    .str.contains("in vitro")
)

# Convert Temperature a numérico
df3[col] = pd.to_numeric(df3[col], errors="coerce")

# ---- 1) In Vitro → imputar 37°C ----
mask_invitro_missing = df3["in_vitro_flag"] & df3[col].isna()
df3.loc[mask_invitro_missing, col] = 37.0

# ---- 2) In Vivo → mediana por Species ----
group_medians = df3.groupby("Species")[col].median()

mask_missing = df3[col].isna()
df3.loc[mask_missing, col] = df3.loc[mask_missing, "Species"].map(group_medians)

# ---- 3) Fallback global ----
global_median = df3[col].median(skipna=True)
df3[col] = df3[col].fillna(global_median)

print("✔ Temperature imputado")
print("Mediana global usada:", global_median)
print("NAs restantes:", df3[col].isna().sum())



✔ Temperature imputado
Mediana global usada: 23.0
NAs restantes: 0


In [20]:
col = "Selected Dose"

# Forzar numérico
df3[col] = pd.to_numeric(df3[col], errors='coerce')

# 1) Imputar SOLO por modo dentro de cada DOI
df3[col] = df3.groupby("DOI")[col].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
)

# 2) Si sigue NA → lo dejamos NA (no inventamos dosis)
print("Remaining NA:", df3[col].isna().sum())


Remaining NA: 1546


In [21]:
# Normalizar columna de tipo de estudio
df3["In vitro/in vivo"] = df3["In vitro/in vivo"].astype(str).str.strip()

# Convertir valores no exactos a forma estándar
df3["mode_flag"] = df3["In vitro/in vivo"].str.lower().str.contains("in vitro")

# Paso 1: In Vitro → not_applicable
mask_invitro = df3["mode_flag"]
df3.loc[mask_invitro, "Sex"] = df3.loc[mask_invitro, "Sex"].replace(
    ["nan", "", "None", "not_reported"], np.nan
)
df3.loc[mask_invitro & df3["Sex"].isna(), "Sex"] = "not_applicable"

# Paso 2: In Vivo → unknown donde falte
mask_invivo = ~df3["mode_flag"]
df3.loc[mask_invivo, "Sex"] = df3.loc[mask_invivo, "Sex"].replace(
    ["nan", "", "None", "not_reported"], np.nan
)
df3.loc[mask_invivo & df3["Sex"].isna(), "Sex"] = "unknown"

print(df3["Sex"].value_counts(dropna=False))
print("✔ Sex imputado igual que en dataset humano")


Sex
M                 1649
not_applicable    1094
F                  965
M/F                666
unknown            208
Name: count, dtype: int64
✔ Sex imputado igual que en dataset humano


In [22]:
# Particle Mass (mg) — imputación por Polymer con fallback global

col = "Particle Mass (mg)"
group = "Polymer"

# Asegurar numérico
df3[col] = pd.to_numeric(df3[col], errors="coerce")
df3[group] = df3[group].astype(str).str.strip()

# Flag missing original
df3["particle_mass_missing"] = df3[col].isna().astype(int)

# Mediana por Polymer
median_by_polymer = df3.groupby(group)[col].median()

# Asignar medianas de grupo a cada fila
df3["mass_median_by_poly"] = df3[group].map(median_by_polymer)

# Fallback: mediana global
global_median = df3[col].median()

# Imputación
df3[col] = df3[col].fillna(df3["mass_median_by_poly"])  # prioridad mediana por polymer
df3[col] = df3[col].fillna(global_median)               # fallback

# Flags de trazabilidad
df3["particle_mass_imputed_by_group"] = (
    (df3["particle_mass_missing"] == 1) & df3["mass_median_by_poly"].notna()
).astype(int)

df3["particle_mass_imputed_by_global"] = (
    (df3["particle_mass_missing"] == 1) & df3["mass_median_by_poly"].isna()
).astype(int)

# Limpiar auxiliares
df3 = df3.drop(columns=["mass_median_by_poly"])

print("=== Imputación Particle Mass (mg) ===")
print("Missing iniciales:", df3["particle_mass_missing"].sum())
print("Imputadas por Polymer:", df3["particle_mass_imputed_by_group"].sum())
print("Imputadas por mediana global:", df3["particle_mass_imputed_by_global"].sum())
print("Missing finales:", df3[col].isna().sum())


=== Imputación Particle Mass (mg) ===
Missing iniciales: 385
Imputadas por Polymer: 374
Imputadas por mediana global: 11
Missing finales: 0


In [23]:
vol_col = "Particle Volume (μm^3)"
size_col = "Size Category"

# Asegurar formatos correctos
df3[size_col] = df3[size_col].astype(str).str.strip().replace({"nan": np.nan})
df3[vol_col] = pd.to_numeric(df3[vol_col], errors="coerce")

# Flag missing original
df3["particle_volume_missing"] = df3[vol_col].isna().astype(int)

# Mediana por Size Category
medians_by_size = df3.groupby(size_col)[vol_col].median()

# Mapear mediana a cada fila
df3["vol_median_by_size"] = df3[size_col].map(medians_by_size)

# Fallback: mediana global
global_median = df3[vol_col].median()

# Imputación
df3[vol_col] = df3[vol_col].fillna(df3["vol_median_by_size"])
df3[vol_col] = df3[vol_col].fillna(global_median)

# Crear flag de imputación por grupo vs global
df3["volume_imputed_by_group"] = (
    (df3["particle_volume_missing"] == 1) & df3["vol_median_by_size"].notna()
).astype(int)

df3["volume_imputed_by_global"] = (
    (df3["particle_volume_missing"] == 1) & df3["vol_median_by_size"].isna()
).astype(int)

# Limpiar auxiliares
df3 = df3.drop(columns=["vol_median_by_size"])

# Reporte
print("=== Imputación Particle Volume (µm³) ===")
print("Missing iniciales:", df3["particle_volume_missing"].sum())
print("Imputadas por Size Category:", df3["volume_imputed_by_group"].sum())
print("Imputadas por mediana global:", df3["volume_imputed_by_global"].sum())
print("Missing finales:", df3[vol_col].isna().sum())


=== Imputación Particle Volume (µm³) ===
Missing iniciales: 359
Imputadas por Size Category: 340
Imputadas por mediana global: 19
Missing finales: 0


In [24]:
col = "Target Organelle, Cell, or Tissue"

# Convertir a string limpio
df3[col] = df3[col].astype(str).str.strip()

# Reemplazar "nan" textual por NaN real
df3[col] = df3[col].replace("nan", np.nan)

# Imputar NA con not_reported
df3[col] = df3[col].fillna("not_reported")

print(df3[col].value_counts(dropna=False).head(20))
print("✔ Target Organelle imputado como 'not_reported'")


Target Organelle, Cell, or Tissue
serum            808
liver            540
colon            372
testis           262
ileum            186
duodenum         166
blood            166
jejunum          152
not_reported     148
caco2.cells      138
a549.cells       100
spleen            95
heart             89
raw2647.cells     85
ovary             84
hela.cells        73
intestine         57
leukocytes        55
hdf.cells         53
sperm             53
Name: count, dtype: int64
✔ Target Organelle imputado como 'not_reported'


In [25]:
col = "Weathered or Biofouled?"

# limpiar strings
df3[col] = df3[col].astype(str).str.strip().str.lower()

# convertir nan textual a NaN real
df3[col] = df3[col].replace("nan", np.nan)

# valores permitidos
valid = ["yes", "no"]

# imputación de NA → unknown
df3[col] = df3[col].where(df3[col].isin(valid), np.nan)
df3[col] = df3[col].fillna("unknown")

print(df3[col].value_counts(dropna=False))
print("✔ Weathered or Biofouled? imputado con 'unknown'")


Weathered or Biofouled?
no         4315
yes         172
unknown      95
Name: count, dtype: int64
✔ Weathered or Biofouled? imputado con 'unknown'


In [26]:
col_cat = "Density, reported or estimated"
col_num = "Density (g/cm^3)"

# 0) Seguridad: asegurar que las columnas existen
assert col_cat in df3.columns, f"{col_cat} no existe en el dataset"
assert col_num in df3.columns, f"{col_num} no existe en el dataset"

# 1) Normalizar strings
df3[col_cat] = df3[col_cat].astype(str).str.strip().str.lower()
df3[col_cat] = df3[col_cat].replace({"nan": np.nan})

# 2) Unificar variantes (por si existen abreviaturas)
df3[col_cat] = df3[col_cat].replace({
    "est": "estimated",
    "estim": "estimated",
    "rep": "reported",
    "rpt": "reported"
})

# 3) Inferir "reported" si:
#    - NO hay categoría
#    - pero SÍ hay valor numérico en Density (g/cm³)
mask_infer_reported = df3[col_cat].isna() & df3[col_num].notna()
df3.loc[mask_infer_reported, col_cat] = "reported"

# 4) El resto de NaN → imputar a "unknown"
df3[col_cat] = df3[col_cat].fillna("unknown")

# 5) Crear flags útiles
df3["density_reported_flag"]  = (df3[col_cat] == "reported").astype(int)
df3["density_estimated_flag"] = (df3[col_cat] == "estimated").astype(int)
df3["density_unknown_flag"]   = (df3[col_cat] == "unknown").astype(int)

# 6) Convertir a categoría
df3[col_cat] = df3[col_cat].astype("category")

# 7) Informe
print(df3[col_cat].value_counts(dropna=False))
print("\nFlags summary:")
print(df3[["density_reported_flag",
         "density_estimated_flag",
         "density_unknown_flag"]].sum())


Density, reported or estimated
estimated    3551
reported     1024
unknown         7
Name: count, dtype: int64

Flags summary:
density_reported_flag     1024
density_estimated_flag    3551
density_unknown_flag         7
dtype: int64


In [27]:
col = "Particle Length (μm)"
shape_col = "Shape"

# Normalizar Shape
df3[shape_col] = df3[shape_col].astype(str).str.strip()
df3[shape_col] = df3[shape_col].replace({"Not Reported": np.nan, "nan": np.nan})

# Crear flag de missing
df3["particle_length_missing"] = df3[col].isna().astype(int)

# Calcular medianas
median_global = df3[col].median()
median_by_shape = df3.groupby(shape_col)[col].median()

# Función de imputación
def impute_length(row):
    val = row[col]
    if pd.notna(val):
        return val
    shape = row[shape_col]
    # si shape tiene mediana válida
    if pd.notna(shape) and pd.notna(median_by_shape.get(shape)):
        return median_by_shape[shape]
    # fallback
    return median_global

# Aplicar
df3[col] = df3.apply(impute_length, axis=1)

# Comprobación
print("Missing después:", df3[col].isna().sum())
print("Mediana global utilizada como fallback:", median_global)


Missing después: 0
Mediana global utilizada como fallback: 1.05


In [28]:
col = "Exposure Media"

# limpiar formato
df3[col] = df3[col].astype(str).str.strip()
df3[col] = df3[col].replace("nan", np.nan)

# imputación
df3[col] = df3[col].fillna("not_reported")

print(df3[col].value_counts(dropna=False).head(20))
print("✔ Exposure Media imputado como 'not_reported'")


Exposure Media
drinking water                         1379
distilled water                         759
RO water                                252
deionized water                         227
culture media                           187
DMEM                                    172
Corn Oil                                172
dulbecco modified eagle medium          152
phosphate buffer saline                 150
RPMI 1610 medium                        115
blood                                    70
water                                    68
double distilled water                   66
Milli-Q water                            63
pure water                               60
basal feed                               60
culture medium DMEM                      51
culture medium                           45
minimum essential media                  40
dulbecco modified eagle medium F-12      39
Name: count, dtype: int64
✔ Exposure Media imputado como 'not_reported'


In [29]:
col = "Replicates"

# convertir a número por seguridad
df3[col] = pd.to_numeric(df3[col], errors="coerce")

# imputación con la moda global
rep_mode = df3[col].mode()[0]
df3[col] = df3[col].fillna(rep_mode)

print(f"✔ Replicates imputado con la moda global: {rep_mode}")
print(df3[col].value_counts().sort_index())


✔ Replicates imputado con la moda global: 1.0
Replicates
1.0     3445
2.0      142
3.0      755
4.0       41
6.0      109
8.0       84
10.0       3
30.0       3
Name: count, dtype: int64


In [30]:
col = "Density (g/cm^3)"

# asegurar numérico
df3[col] = pd.to_numeric(df3[col], errors="coerce")

# calcular mediana global
median_density = df3[col].median()

# imputación
df3[col] = df3[col].fillna(median_density)

print(f"✔ Density (g/cm³) imputado con la mediana global: {median_density}")
print("NAs restantes:", df3[col].isna().sum())


✔ Density (g/cm³) imputado con la mediana global: 1.07
NAs restantes: 0


In [31]:
col = "Sample Size"
group = "Experiment Type"

# 1) Asegurar numérico
df3[col] = pd.to_numeric(df3[col], errors="coerce")

# 2) Flag de missing (trazabilidad)
df3["sample_size_missing"] = df3[col].isna().astype(int)

# 3) Imputación por mediana dentro de cada Experiment Type
df3[col] = df3.groupby(group)[col].transform(
    lambda x: x.fillna(x.median())
)

# 4) Fallback: mediana global
global_median = df3[col].median()
df3[col] = df3[col].fillna(global_median)

# 5) Convertir a entero solo si todos los valores son enteros
if df3[col].dropna().apply(float.is_integer).all():
    df3[col] = df3[col].astype("Int64")

print("✔ Sample Size imputado por medianas por grupo")
print("Mediana global fallback:", global_median)
print("NAs restantes:", df3[col].isna().sum())
print("Valores únicos finales:", sorted(df3[col].unique()))


✔ Sample Size imputado por medianas por grupo
Mediana global fallback: 5.0
NAs restantes: 0
Valores únicos finales: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(5.5), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0), np.float64(10.0), np.float64(12.0), np.float64(12.5), np.float64(15.0), np.float64(18.0), np.float64(20.0)]


In [32]:
#nulos
with pd.option_context('display.max_rows', None):
    display(df3.isna().mean().sort_values(ascending=False) * 100)

Selected Dose                           33.740725
Shape                                    6.765605
Species                                  0.000000
Life Stage                               0.000000
DOI                                      0.000000
Year                                     0.000000
Sex                                      0.000000
In vitro/in vivo                         0.000000
Particle Mix?                            0.000000
Experiment Type                          0.000000
Reference Particle                       0.000000
Exposure Media                           0.000000
Solvent                                  0.000000
Exposure Route                           0.000000
Detergent                                0.000000
Temperature (Avg)                        0.000000
Number of Doses                          0.000000
Exposure Duration (days)                 0.000000
Replicates                               0.000000
Sample Size                              0.000000


In [33]:
df3['Polymer'].unique()

array(['Polystyrene', 'Polyethylene', 'Polymethylmethacrylate',
       'Polyvinylchloride', 'Not Reported', 'Polypropylene', 'Polyamide',
       'Tire Rubber', 'Polyurathane'], dtype=object)

In [34]:
df3.shape

(4582, 70)

In [35]:
df3_processed = df3
df3_processed.to_csv("../data/processed/human_summary_processed.csv", index=False)

In [36]:
df3_processed.shape

(4582, 70)

In [37]:
df = df3_processed.copy()   # tu dataset SUMMARY procesado

print("Número total de filas:", len(df))
print("Número de DOIs únicos:", df["DOI"].nunique())

# --- Función para contar valores únicos dentro de cada DOI ---
def count_unique_in_group(col):
    return df.groupby("DOI")[col].nunique(dropna=False)

# --- Analizar todas las columnas excepto DOI ---
results = []

for col in df.columns:
    if col == "DOI":
        continue

    unique_counts = count_unique_in_group(col)
    max_unique = unique_counts.max()
    num_dois_changing = (unique_counts > 1).sum()

    results.append({
        "column": col,
        "max_unique_within_DOI": max_unique,
        "DOIs_where_column_varies": num_dois_changing
    })

# Convertimos a DataFrame
analysis_df = pd.DataFrame(results).sort_values(
    "DOIs_where_column_varies", ascending=False
)

print("\n========= RESULTADOS DEL ANÁLISIS =========\n")
print(analysis_df)

# Mostrar solo columnas que NO son constantes dentro del DOI
print("\n========= COLUMNAS QUE VARÍAN DENTRO DE DOI =========\n")
print(analysis_df[analysis_df["DOIs_where_column_varies"] > 0])

# Mostrar solo columnas que SON constantes dentro del DOI
print("\n========= COLUMNAS CONSTANTES DENTRO DE DOI =========\n")
print(analysis_df[analysis_df["DOIs_where_column_varies"] == 0])


Número total de filas: 4582
Número de DOIs únicos: 65

========= RESULTADOS DEL ANÁLISIS =========

                                 column  max_unique_within_DOI  \
23                             Endpoint                    101   
22           Specific Endpoint Category                     18   
21              Broad Endpoint Category                      8   
20                               Effect                      2   
24     Level of Biological Organization                      4   
..                                  ...                    ...   
47  Background Contamination Monitored?                      1   
40                   Polymer Validated?                      1   
59       particle_mass_imputed_by_group                      1   
62              volume_imputed_by_group                      1   
68                  sample_size_missing                      1   

    DOIs_where_column_varies  
23                        61  
22                        59  
21            